In [1]:
pip install nmslib

Note: you may need to restart the kernel to use updated packages.


In [3]:
from google.colab import drive
from sklearn.linear_model import LogisticRegression
import pickle
import numpy as np
from math import sqrt
import numpy
import sys
import nmslib
import time
import math
import pdb

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
cd gdrive
cd My\ Drive
cd eurlex4k/

In [4]:
class DataSet:
  def __init__(self,x_train_name,y_train_name):
    self.x_train_name = x_train_name
    self.y_train_name = y_train_name
    self.X_train = self.read_X_dense(x_train_name)
    self.Y_train = self.read_Y(y_train_name)
  def read_X_dense(self,file_name):
    with open(file_name) as reader:
      dimension_line = reader.readline()
      self.no_of_data_points = int(dimension_line.split(" ")[0])
      self.no_of_features = int(dimension_line.split(" ")[1])
      data = np.zeros((self.no_of_data_points,self.no_of_features))
      row_number = 0
      while True: 
        line = reader.readline() 
        if not line: 
            break
        column_number = 0
        for x in line.split(" "):
          data[row_number][column_number] = float(x)
          column_number+=1
        row_number+=1
      return data
  def read_Y(self,file_name):
    with open(file_name) as reader:
      dimension_line = reader.readline()
      no_of_data_points = int(dimension_line.split(" ")[0])
      self.no_of_labels = int(dimension_line.split(" ")[1])
      data = []
      while True: 
        line = reader.readline() 
        if not line: 
            break
        column_number = 0
        data_row=[]
        for x in line.split(" "):
          data_row.append(int(x.split(":")[0]))
        data.append(data_row)
      return data
  def clean_dataset(self):
    remove_training_points_with_no_features()
    remove_labels_with_no_training_data()
    return
  def remove_training_points_with_no_features(self):
    final_X=[]
    final_Y=[]
    for i in range(self.no_of_data_points):
      norm = 0
      for j in range(self.no_of_features):
        norm+=(self.X_train[i][j]**2.0)
      if norm==0:
        continue
      final_X.append(self.X_train[i])
      final_Y.append(self.Y_train[i])
    self.X_train = np.array(final_X)
    self.Y_train = final_Y

In [ ]:
dataset = DataSet('trn_ft_mat_dense.txt','trn_lbl_mat.txt')
dataset.remove_training_points_with_no_features() #may be clean_dataset()

In [ ]:
class DummyModel:
  def __init__(self,val):
    self.val = val
    self.coef_ = np.array([[0]])
    self.intercept_ = np.array([val])
  def predict_proba(y):
    return np.full(len(y), val)


In [ ]:
class HNSW:
  def __init__(self,training_data,params):
    self.training_data = training_data
    self.M = params.param.M
    self.efC = params.param.efC
    self.efS = params.param.efS
    self.metric_space = "cosinesimil"
    self.num_threads = params.param.num_threads
    self.num_nbrs = params.param.num_nbrs
  def train(self,path):
    self.index = nmslib.init(method='hnsw', space=self.metric_space)
    self.index.addDataPointBatch(self.training_data)
    self.index.createIndex({'M': self.M, 'indexThreadQty': self.num_threads, 'efConstruction': self.efC})
    nmslib.saveIndex(self.index,path)

  def find_closest_neighbours(self,data,label_to_datapoint_index):
    self.index.setQueryTimeParams({'efSearch': self.efS, 'algoType': 'old'})
    nbrs = self.index.knnQueryBatch(data, k=self.num_nbrs, num_threads = self.num_threads)
    for i in range(len(data)):
      for j in range(self.num_nbrs):
        label_to_datapoint_index[nbrs[i][0][j]].append(i)
  def load(self,model_file):
    self.index = nmslib.init(method='hnsw', space=self.metric_space)
    nmslib.loadIndex(self.index,model_file)
  def evaluate_generative_model(self,data,model_file):
    self.load(model_file)
    self.index.setQueryTimeParams({'efSearch': self.efS, 'algoType': 'old'})
    nbrs = self.index.knnQueryBatch(data, k=self.num_nbrs, num_threads = self.num_threads)
    arr = []
    for row in nbrs:
      arr.append(row[0]) 
    return np.array(arr)
    
        
